# redis-py

In [1]:
import redis

In [2]:
client = redis.Redis(decode_responses=True)

## store and retrieve

### a k:v string pair

In [3]:
client.set(name="foo", value="bar")

True

In [4]:
client.get(name="foo")

'bar'

### a dict

In [5]:
client.hset(name="username:123", # hash name
       mapping={
            "name": "John",
            "surname": 'Smith',
            "company": 'Redis',
            "age": 29
       }
       )

0

In [6]:
client.hgetall(name="username:123")

{'name': 'John', 'surname': 'Smith', 'company': 'Redis', 'age': '29'}

# Redis as a vector db

[ref](https://redis.io/docs/latest/develop/get-started/vector-database/#install-the-required-python-packages)

In [7]:
import json

In [8]:
with open("../data/wikihowAll.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [9]:
from pprint import pprint

In [10]:
pprint(json.dumps(data[0][1], indent=4))

('{\n'
 '    "headline": "\\nCreate a sketch in the NeoPopRealist manner of the '
 'future mural on a small piece of paper 8\\"x10\\" using the black ink '
 'pen.,\\nPrepare to create your NeoPopRealist mural.,\\nPrepare your '
 'paint.,\\nBegin your project with a design.,\\nProduce a scaled down version '
 'of your finished mural.,\\nPrepare the wall to be painted.,\\nAfter you have '
 'primed the surface, measure the wall.,\\nPaint in the base coat of the '
 'background.,\\nAllow the background and base coats to dry.,\\nDraw the '
 'lines, then fill the appeared section with different repetitive patterns '
 '(examine the images above).,\\nPaint patterns with brushes of suitable size '
 'for the particular portion of work you are painting.,\\nClean up the lines '
 'and shapes as needed.,\\nSeal the mural if needed.,\\nBe inspired and it '
 'will help you succeed!",\n'
 '    "title": "How to Create a Neopoprealist Art Work",\n'
 '    "text": " See the image for how this drawing develo

In [11]:
pipeline = client.pipeline()

In [12]:
for i, article in enumerate(data[0]):
    redis_key = f"article_{i:03}:{article["title"]}"
    pipeline.json().set(name=redis_key, path="$", obj=article)

In [13]:
res = pipeline.execute()

In [16]:
len(res) # 996 articles were loaded

996

In [24]:
res = client.json().get("article_001:How to Create a Neopoprealist Art Work")
res

{'headline': '\nCreate a sketch in the NeoPopRealist manner of the future mural on a small piece of paper 8"x10" using the black ink pen.,\nPrepare to create your NeoPopRealist mural.,\nPrepare your paint.,\nBegin your project with a design.,\nProduce a scaled down version of your finished mural.,\nPrepare the wall to be painted.,\nAfter you have primed the surface, measure the wall.,\nPaint in the base coat of the background.,\nAllow the background and base coats to dry.,\nDraw the lines, then fill the appeared section with different repetitive patterns (examine the images above).,\nPaint patterns with brushes of suitable size for the particular portion of work you are painting.,\nClean up the lines and shapes as needed.,\nSeal the mural if needed.,\nBe inspired and it will help you succeed!',
 'title': 'How to Create a Neopoprealist Art Work',
 'text': ' See the image for how this drawing develops step-by-step. However, there is an important detail: the following drawings are to exam

# Embedding model

use huggingface's `SentenceTransformer` framework to use locally servable embeddding models

In [25]:
from sentence_transformers import SentenceTransformer

2025-03-24 22:39:45.778090: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Choose embedding model

[ref](https://www.pinecone.io/learn/series/rag/embedding-models-rundown/)

Use [`NV-Embed-v2`](https://huggingface.co/nvidia/NV-Embed-v2)

It is the highest ranking opesource model on MTEB leaderboard.

In [ ]:
embedding_model = SentenceTransformer(
    model_name_or_path="nvidia/NV-Embed-v2",
    trust_remote_code=True
)